In [ ]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Import regular expressions to process emails
import re

# Plotting library
from matplotlib import pyplot as plt

# Optimization module in scipy
from scipy import optimize

# will be used to load MATLAB mat datafile format
from scipy.io import loadmat

# the utility functions
import utils

from sklearn import svm

# tells matplotlib to embed plots within the notebook
%matplotlib inline

In [ ]:
kenel_method = 'poly'
X = train_datasets
Y = train_labels
# model = svm.SVC(kernel = kenel_method)
model = svm.LinearSVC()
dec = model.fit(X, Y)

In [ ]:
from sklearn.cluster import KMeans
X = train_datasets
kmeans = KMeans(n_clusters=11, random_state=0).fit(X)

In [ ]:
centers = kmeans.cluster_centers_
center_index = []
for center in centers:
    for i,sample in enumerate(train_datasets):
        if (sum(center == sample) == len(center)):
            center_index.append(i)

In [ ]:
prediction = kmeans.predict(test_datasets)
sum(prediction == test_labels.reshape(test_data_num)) / test_labels.shape[0]

In [ ]:
dec.score(train_datasets, train_labels)

In [ ]:
test_data_num = m['test_pattern'].shape[0]
test_datasets = np.zeros((test_data_num,input_neuron_num,max_time))
for i in range(train_data_num):
    current_array = m['test_pattern'][i][0]
    current_spikes_input = np.zeros((max_time, input_neuron_num))
    for input_idx in range(current_array.shape[0]):
        for spike_time in current_array[input_idx]:
            if spike_time < max_time:
                current_spikes_input[int(spike_time)][input_idx] = 1
    test_datasets[i] = current_spikes_input.T
test_datasets = test_datasets.reshape(test_data_num,-1)
test_labels = m['test_labels']-1

In [ ]:
dec.score(test_datasets, test_labels)

In [ ]:
dec.predict(test_datasets)

In [ ]:
prediction[:100]

In [ ]:
sum(prediction == test_labels.reshape(test_data_num)) / test_labels.shape[0]

In [87]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class baseline_linear(nn.Module):
    def __init__(self,input_neuron_num,max_time,num_class,device):
        super(baseline_linear, self).__init__()
        self.fc = nn.Linear(input_neuron_num * max_time, num_class)
        self.device = device
        self.activation = nn.Softmax()
        
    def forward(self, samples):
        bsz = samples.shape[0]
        samples = samples.reshape(bsz,-1).to(self.device)
        out = self.fc(samples)
#         out = self.activation(out)
        
        return out
    
    def init_param(self):
        for param in self.fc.parameters():
            torch.nn.init.normal_(param)
    

class baseline_rnn(nn.Module):
    def __init__(self,input_neuron_num,max_time,num_class,device):
        super(baseline_rnn, self).__init__()
        self.device = device
        self.input_neuron_num = input_neuron_num
        self.rnn_list = [nn.LSTM(input_size = 1, hidden_size = 1, num_layers = 1, batch_first = True).to(device) for i in range(num_class)]
#         self.rnn = nn.RNN(input_size = num_class )
        self.fc = nn.Linear(input_neuron_num,num_class).to(device)
        self.activation = nn.Sigmoid()
        self.max_time = max_time
        self.num_class = num_class
        
    def forward(self,samples):
        bsz = samples.shape[0]
        ## (bsz, max_time, input_neuron_num)
#         out = torch.FloatTensor(bsz, self.max_time, self.num_class)
#         for step in range(self.max_time):
#             out[:,step] = self.fc(samples[:,step])
        out = self.fc(samples)
#         print(out.shape)
            
        out = out.permute(0,2,1)
        ## (bsz, num_class, max_time)
        out2 = torch.FloatTensor(bsz, self.num_class).to(self.device)
        for i in range(self.num_class):
            out2[:,i] = self.rnn_list[i](out[:,:,i].unsqueeze(-1))[1][0][0].squeeze(1)
#         print(out2.shape)
        return self.activation(out2)

    def init_param(self):
        for rnn in self.rnn_list:
            for param in rnn.parameters():
                torch.nn.init.normal_(param)


class baseline_rnn2(nn.Module):
    def __init__(self,input_neuron_num,max_time,num_class,device):
        super(baseline_rnn, self).__init__()
        self.device = device
        self.input_neuron_num = input_neuron_num
        self.rnn_list = [nn.RNN(input_size = 1, hidden_size = 1, num_layers = 1, batch_first = True).to(device) for i in range(input_neuron_num)]
#         self.rnn = nn.RNN(input_size = num_class )
        self.fc = nn.Linear(input_neuron_num,num_class)
        self.activation = nn.Sigmoid()
        self.max_time = max_time
        self.num_class = num_class
        
    def forward(self,samples):
        bsz = samples.shape[0]
        ## (bsz, max_time, input_neuron_num)
        out = torch.FloatTensor(bsz, self.input_neuron_num)
        for i in range(self.input_neuron_num):
            out[:,i] = self.self.rnn_list[i](samples[:,:,step])[1][0]
            
        out = self.fc(out)
        out = self.activation(out)
        

    def init_param(self):
        for rnn in self.rnn_list:
            for param in rnn.parameters():
                torch.nn.init.normal_(param)
            

class baseline_rnn3(nn.Module):
    def __init__(self,input_neuron_num,max_time,num_class,device):
        super(baseline_rnn2, self).__init__()
        self.device = device
        self.input_neuron_num = input_neuron_num
        self.rnn = nn.RNN(input_size = input_neuron_num, hidden_size = num_class, batch_first = True)
        
        self.activation = nn.Sigmoid()
        
        self.max_time = max_time
        self.num_class = num_class
        
    def forward(self,samples):
        bsz = samples.shape[0]
#         print(samples.shape)
        ## (bsz, max_time, input_neuron_num)
        out = self.rnn(samples)[1]
        out = self.activation(out).squeeze(1)
        
        return out
    
    def init_param(self):
        for param in self.rnn.parameters():
            torch.nn.init.normal_(param)

In [34]:
##### Load Train Dataset ##### 
import torch
from scipy.io import loadmat 
import numpy as np
m = loadmat("Spike-TIDIGITS/Spike-TIDIGITS.mat")
max_time = 130 #ms
input_neuron_num = 620
input_comb_coef = 10
train_data_num = m['train_pattern'].shape[0]
train_datasets = np.zeros((train_data_num, max_time, int(input_neuron_num/1)))
for i in range(train_data_num):
    current_array = m['train_pattern'][i][0]
    current_spikes_input = np.zeros((max_time, int(input_neuron_num/1)))
    for input_idx in range(current_array.shape[0]):
        for spike_time in current_array[input_idx]:
            if spike_time < max_time:
                current_spikes_input[int(spike_time)][int(input_idx/1)] = 1
    train_datasets[i] = current_spikes_input
train_datasets = torch.Tensor(train_datasets).byte()
train_labels = torch.LongTensor(m['train_labels']-1)



##### Load Test Dataset ##### 
# m = loadmat("./Spike TIDIGITS/Spike-TIDIGITS.mat")
# max_time = 50 #ms
# input_neuron_num = 620
test_data_num = m['test_pattern'].shape[0]
test_datasets = np.zeros((test_data_num, max_time, input_neuron_num))
for i in range(train_data_num):
    current_array = m['test_pattern'][i][0]
    current_spikes_input = np.zeros((max_time, input_neuron_num))
    for input_idx in range(current_array.shape[0]):
        for spike_time in current_array[input_idx]:
            if spike_time < max_time:
                current_spikes_input[int(spike_time)][input_idx] = 1
    test_datasets[i] = current_spikes_input
test_datasets = torch.Tensor(test_datasets).byte()
test_labels = torch.LongTensor(m['test_labels']-1)
n_test = test_labels.shape[0]

In [35]:
datasets = torch.utils.data.TensorDataset(test_datasets,test_labels)
train_loader = torch.utils.data.DataLoader(datasets, batch_size = 8, shuffle=True, drop_last = True)

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jun  9 09:46:25 2018

@author: yjwu

Python 3.5.2

"""

from __future__ import print_function
import torchvision
import torchvision.transforms as transforms
import os
import time
# os.environ['CUDA_VISIBLE_DEVICES'] = "3"
# names = 'spiking_model'
# data_path =  './raw/' #todo: input your data path
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# train_dataset = torchvision.datasets.MNIST(root= data_path, train=True, download=True, transform=transforms.ToTensor())
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# test_set = torchvision.datasets.MNIST(root= data_path, train=False, download=True,  transform=transforms.ToTensor())
# test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0)

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
acc_record = list([])
loss_train_record = list([])
loss_test_record = list([])

device = torch.device("cuda:0")
n_classes = 11
learning_rate = 1e-3
num_epochs = 10000
batch_size = 8

snn = baseline_linear(620,max_time,n_classes,device).to(device)
snn.init_param()
# snn = snn.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(snn.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    running_loss = 0
    start_time = time.time()
    snn.train()
    for step, batch in enumerate(train_loader):
        input_data,labels = batch
#         print(input_data.shape)
        
#         labels = train_labels[step]
    
#         snn.zero_grad()
        optimizer.zero_grad()

        input_data = input_data.float().to(device)
#         input_data = input_data.unsqueeze(0)
#         print(input_data.shape)
        
        outputs = snn(input_data)
        
#         labels_ = torch.zeros(batch_size, n_classes).scatter_(1, labels.view(-1, 1), 1).to(device)
        labels = labels.to(device).squeeze(1)
#         print(outputs.device)
#         print(labels_.device)
        loss = criterion(outputs, labels)
#         print(loss.item())
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        if (step+1)%100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.5f'
                    %(epoch+1, num_epochs, step+1, train_datasets.shape[0]//batch_size,running_loss))
            running_loss = 0
            print('Time elasped:', time.time()-start_time)
    correct = 0
    total = 0
#     optimizer = lr_scheduler(optimizer, epoch, learning_rate, 40)

    with torch.no_grad():
        snn.eval()
        for step, test_data in enumerate(test_datasets):
            test_label = test_labels[step]
            test_data = test_data.float().to(device).unsqueeze(0)
                                                              
            outputs = snn(test_data)

            predicted = outputs.cpu().argmax(1)
            
#             print(outputs.shape)
#             print(predicted)
#             print(test_label)
            total += float(test_data.size(0))
            correct += float(predicted.eq(test_label).sum().item())
#             if step %100 ==0:
        acc = 100. * float(correct) / float(total)
        print(step, test_datasets.shape[0],' Acc: %.5f' % acc)

#     print('Iters:', epoch,'\n\n\n')
#     print('Test Accuracy of the model on the 10000 test images: %.3f' % (100 * correct / total))
#     acc = 100. * float(correct) / float(total)
#     acc_record.append(acc)
#     if epoch % 5 == 0:
#         print(acc)
#         print('Saving..')
#         state = {
#             'net': snn.state_dict(),
#             'acc': acc,
#             'epoch': epoch,
#             'acc_record': acc_record,
#         }
#         if not os.path.isdir('checkpoint'):
#             os.mkdir('checkpoint')
#         torch.save(state, './checkpoint/ckpt' + names + '.t7')
#         best_acc = acc

/home/nesa320/anaconda2/envs/contras_sum/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Epoch [1/10000], Step [100/308], Loss: 245.04502
Time elasped: 1.9587175846099854
Epoch [1/10000], Step [200/308], Loss: 243.49759
Time elasped: 3.1542325019836426
Epoch [1/10000], Step [300/308], Loss: 245.07730
Time elasped: 5.446819067001343
2485 2486  Acc: 12.51006
Epoch [2/10000], Step [100/308], Loss: 240.94674
Time elasped: 0.9597737789154053
Epoch [2/10000], Step [200/308], Loss: 240.86934
Time elasped: 2.0387260913848877
Epoch [2/10000], Step [300/308], Loss: 240.30074
Time elasped: 5.230144262313843
2485 2486  Acc: 15.60740
Epoch [3/10000], Step [100/308], Loss: 238.74029
Time elasped: 2.7577056884765625
Epoch [3/10000], Step [200/308], Loss: 236.98014
Time elasped: 5.018935203552246
Epoch [3/10000], Step [300/308], Loss: 237.27839
Time elasped: 7.865734100341797
2485 2486  Acc: 17.65889
Epoch [4/10000], Step [100/308], Loss: 234.82802
Time elasped: 1.7394917011260986
Epoch [4/10000], Step [200/308], Loss: 237.35396
Time elasped: 3.5434134006500244
Epoch [4/10000], Step [300/